<h1 style="text-align:center;">Getting BIOS Settings</h1>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook demonstrates an example of retrieving bios settings from iLO servers. It utilizes the HPE iLOrest library to interact with the iLO API and retrieve the necessary information.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
    </ul>
</div>

In [7]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

#This is to take passwords as input securely
import getpass

<div class = "alert alert-block alert-success" style="border-radius:20px;">
    <div style = "color:green; font-size:40px;"></div><br>
    <h3>2. Below is the get_bios_settings function, after everything is done this function will give details about bios settings of system.</h3><br>
    <i>Let me give you a walk through of it!</i>
    <br>
    There is a hierarchy involved in iLO Rest Api and everything is in form of <a href = "https://www.w3schools.com/whatis/whatis_json.asp" style = "background-color:powderblue;">JSON</a> objects i.e. name-value pairs. So we are simply following the hierarchy to navigate to required information.

1. In the <b>get_bios_setting</b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using <i>get_resource_directory</i> module.
<br>
    
2. In case the resource directory was disabled we are using systems making a series of http <a href = "https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods" style="background-color:powderblue;">GET</a>. requests to the server so as to obtain info about bios settings.
    <ul>
    <li>The information about bios settings is present inside a particular json object of iLO rest api. </li><li>We just need to navigate to that directory so as to obtain info about bios settings.</li> <li>For that purpose simply we are making series of get requests.</li>
        We make a final get operation by at the location <b>bios_uri</b>  and save the response inside <b>bios_data.</b>
    </ul>
    <br>
 

3. Else if resource directory is enabled then we will simply iterate through all instances inside resources 
    and if #bios attribute is present in <b>@odata.type</b> attribute then we'll store information about bios settings of the system inside of bios_data variable and break out of iteration.
    <br>

Ultimately, if bios_data variable is not null we print out information about bios settings. 

 </div>

In [1]:
def get_bios_setting(_redfishobj):

    bios_uri = None
    bios_data = None
    resource_instances = _redfishobj.get_resource_directory()
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        systems_uri = _redfishobj.root.obj['Systems']['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
        systems_members_uri = next(iter(systems_response.obj['Members']))['@odata.id']
        systems_members_response = _redfishobj.get(systems_members_uri)
        bios_uri = systems_members_response.obj['Bios']['@odata.id']
        bios_data = _redfishobj.get(bios_uri)
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#Bios.' in instance['@odata.type']:
                bios_uri = instance['@odata.id']
                bios_data = _redfishobj.get(bios_uri)
                break

    if bios_data:
        print("\n\nShowing bios attributes before changes:\n\n")
        print(json.dumps(bios_data.dict, indent=4, sort_keys=True))

<div class="alert alert-block alert-success" style="border-radius:20px;">
    <div style = "color:green; font-size:40px;"></div><br>
<h3>3. The Main function: </h3><br>
    <ul>
        <li>Once your credentials are entered, a redfish obejct is created with the help of RedfishClient class and a session is created and now you will be logged in to the server.</li>
        <li>If server is down or unreachable, we get server not reachable message!</li>
        <li>If not, then we make a call to <b>get_bios_settings function</b> to obtain information about bios settings of system.</li>
        <li>After getting required information <i><b>Redfish.logout()</b></i>closes the particular session and logs us out of the system.</li>
    </ul>
</div>

<div class = "alert alert-block alert-warning" style="border-radius:20px;">
    <b>When running on the server locally use the following commented values</b><br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

In [2]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    get_bios_setting(REDFISHOBJ)
    REDFISHOBJ.logout()

Enter iLO IP address: 10.0.2.15
Enter Username: Administrator
Enter password: 12iso*help


NameError: name 'ServerDownOrUnreachableError' is not defined

<div class = "alert alert-block alert-success" style="border-radius:20px;">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>